### Codio Assignment 12.2: Decision Boundaries

**Estimated Time: 60 Minutes**

**Total Points: 55**

This activity focuses on the effect of changing your decision threshold and the resulting predictions.  Again, you will use the `KNeighborsClassifier` ,but this time, you will explore the `predict_proba` method of the fit estimator to change the thresholds for classifying observations.  You will explore the results of changing the decision threshold on the false negative rate of the classifier for the insurance data.  Here, we suppose the important thing is to not make the mistake of predicting somebody would not default when they really do.  

#### Index

- [Problem 1](#Problem-1)
- [Problem 2](#Problem-2)
- [Problem 3](#Problem-3)
- [Problem 4](#Problem-4)
- [Problem 5](#Problem-5)
- [Problem 6](#Problem-6)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn import set_config

set_config(display="diagram")

### The Dataset

You continue to use the default example, and the data is again loaded and split for you below. 

In [3]:
default = pd.read_csv('data/default.csv')

In [4]:
default.head(2)

,Unnamed: 0,default,student,balance,income
0,1,No,No,729.526495,44361.625074
1,2,No,Yes,817.180407,12106.134700


In [5]:
X_train, X_test, y_train, y_test = train_test_split(default.drop('default', axis = 1), 
                                                    default['default'],
                                                   random_state=42)

In [6]:
transformer = make_column_transformer((OneHotEncoder(drop = 'if_binary'), ['student']),
                                     remainder = StandardScaler())

[Back to top](#-Index)

### Problem 1

#### Basic Pipeline

**10 Points**

Use the `Pipeline` function to create a pipeline `base_pipe` with steps `transformer` and `knn`. Assign `transformer` to `'transformer'` and assign a `KNeighborsClassifier()` with `n_neighbors = 10` to `'knn'`. 

In [7]:
### GRADED

base_pipe = ''

# YOUR CODE HERE
base_pipe = Pipeline(steps=[
    ('transformer', transformer),  # Add the transformer step
    ('knn', KNeighborsClassifier(n_neighbors=10))  # Add the KNN classifier step
])

# Answer check
base_pipe

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['student'])])),
                ('knn', KNeighborsClassifier(n_neighbors=10))])

[Back to top](#-Index)

### Problem 2

#### Accuracy of KNN with 50% probability boundary

**10 Points**

- Use the `fit` function to train `base_pipe` on `X_train` and `y_train`.
- Use the `score` function to calculate the performance of `base_pipe` on the test sets. Assign the result to `base_acc`.
- Use the `predict` function on `base_pipe` to make predictions on `X_test`. Assign the reusl to `preds`.
- Initialize the `base_fn` variable to `0`.
- Use a `for` loop to loop over `zip(preds, y_test)`. Inside the `for` loop:
    - Use an `if` block to determine the accuracy for this default setting and assign it to `base_acc`. Also, consider the proportion of false negatives here.  Assign these as `base_fn`.  

In [8]:
### GRADED

base_acc = ''
base_fn = ''

# YOUR CODE HERE
# Fit the pipeline on the training data
base_pipe.fit(X_train, y_train)

# Calculate the accuracy of the pipeline on the test set
base_acc = base_pipe.score(X_test, y_test)

# Make predictions on the test set
preds = base_pipe.predict(X_test)

# Initialize the base_fn variable for counting false negatives
base_fn = 0

# Loop over predictions and true values to calculate false negatives
for pred, true in zip(preds, y_test):
    if true == 1 and pred == 0:  # True positive is 1, False negative is when pred = 0 but true = 1
        base_fn += 1

# Answer check
print(base_acc)
print(base_fn)

0.9712
0


[Back to top](#-Index)

### Problem 3

#### Prediction probabilities

**10 Points**

As demonstrated in Video 12.5, your fit estimator has a `predict_proba` method that will output a probability for each observation.  


Use the `predict_proba` function on `base_pipe` to predict the probabilities on `X_test`. Assign the predicted probabilities as an array using the test data to `base_probs` below. 

In [9]:
### GRADED

base_probs = ''

# YOUR CODE HERE
# Generate predicted probabilities for the test set
base_probs = base_pipe.predict_proba(X_test)


# Answer check
pd.DataFrame(base_probs[:5], columns = ['p_no', 'p_yes'])

,p_no,p_yes
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


[Back to top](#-Index)

### Problem 4

#### A Stricter `default` estimation

**10 Points**

As discussed in the previous assignment, if you aim to minimize the number of predictions that miss default observations you may consider increasing the probability threshold to make such a classification.  Accordingly, use your probabilities from the last problem to only predict 'No' if you have a higher than 70% probability that this is the label.  Assign your new predictions as an array to `strict_preds`.  Determine the number of false negative predictions here and assign them to `strict_fn` below.  

In [10]:
### GRADED

strict_fn = ''

# YOUR CODE HERE
# Initialize strict_fn for counting false negatives
strict_fn = 0

# Define a stricter threshold for classification
threshold = 0.9

# Loop through predicted probabilities and true values
for prob, true in zip(base_probs[:, 1], y_test):  # Use only the probabilities for the positive class
    pred_strict = 1 if prob > threshold else 0  # Apply strict threshold
    if true == 1 and pred_strict == 0:  # False negative: true = 1, pred_strict = 0
        strict_fn += 1

# Answer check
print(strict_fn)

0


[Back to top](#-Index)

### Problem 5

#### Minimizing False Negatives

**10 Points**

Consider a 50%, 70%, and 90% decision boundary for predicting "No".  Which of these minimizes the number of false negatives?  Assign your solution as an integer -- 50, 70, or 90 -- to `ans5` below.



In [11]:
### GRADED

ans5 = ''

# YOUR CODE HERE
# Initialize counts of false negatives for each threshold
false_negatives = {}

# Thresholds to test
thresholds = [0.5, 0.7, 0.9]

# Loop through thresholds and count false negatives
for threshold in thresholds:
    fn = 0
    for prob, true in zip(base_probs[:, 1], y_test):  # Use probabilities for the positive class
        pred = 1 if prob > threshold else 0  # Predict based on the current threshold
        if true == 1 and pred == 0:  # False negative condition
            fn += 1
    false_negatives[threshold] = fn

# Find the threshold that minimizes false negatives
ans5 = min(false_negatives, key=false_negatives.get)

# Answer check
print(ans5)

0.5


[Back to top](#-Index)

### Problem 6

#### Visualizing decision boundaries

**5 Points**

For this exercise, a visualization of the decision boundary using a synthetic dataset is created and plotted below.  Which of these would you choose to minimize the number of false negatives?  Enter your choice as an integer -- 1, 20, or 50 -- to `ans6` below.

<center>
    <img src = images/dbounds.png />
</center>

In [12]:
### GRADED

ans6 = ''

# YOUR CODE HERE
# Define potential decision boundary values
decision_boundaries = [1, 20, 50]

# Placeholder to count false negatives for each boundary
false_negatives = {}

# Loop through each decision boundary and compute false negatives
for boundary in decision_boundaries:
    fn = 0
    # Generate predictions using the current boundary
    for prob, true in zip(base_probs[:, 1], y_test):  # Positive class probabilities
        pred = 1 if prob > (boundary / 100) else 0  # Adjust boundary to percentage (e.g., 1% = 0.01)
        if true == 1 and pred == 0:  # Count false negatives
            fn += 1
    false_negatives[boundary] = fn

# Choose the boundary that minimizes false negatives
ans6 = min(false_negatives, key=false_negatives.get)

# Answer check
print(ans6)

1
